# LSTM-arithmetic

## Dataset
- [Arithmetic dataset](https://drive.google.com/file/d/1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-/view?usp=sharing)

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
import os

data_path = './data'

In [2]:
df_train = pd.read_csv(os.path.join(data_path, 'arithmetic_train.csv'))
df_eval = pd.read_csv(os.path.join(data_path, 'arithmetic_eval.csv'))

In [3]:
# transform the input data to string
df_train['tgt'] = df_train['tgt'].apply(lambda x: str(x))
df_train['src'] = df_train['src'].add(df_train['tgt'])
df_train['len'] = df_train['src'].apply(lambda x: len(x))

df_eval['tgt'] = df_eval['tgt'].apply(lambda x: str(x))
df_eval['src'] = df_eval['src'].add(df_eval['tgt'])
df_eval['len'] = df_eval['src'].apply(lambda x: len(x))

In [4]:
df_train.head()

,Unnamed: 0,src,tgt,len
0,2285313,14*(43+20)=882,882,14
1,317061,(6+1)*5=35,35,10
2,718770,13+32+29=74,74,11
3,170195,31*(3-11)=-248,-248,14
4,2581417,24*49+1=1177,1177,12


# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [5]:
char_to_id = {}
id_to_char = {}

# write your code here
# Build a dictionary and give every token in the train dataset an id
# The dictionary should contain <eos> and <pad>
# char_to_id is to conver charactors to ids, while id_to_char is the opposite

char_to_id['<pad>'] = 0
id_to_char[0] = '<pad>'
char_to_id['<eos>'] = 1
id_to_char[1] = '<eos>'
char_to_id['='] = 2
id_to_char[2] = '='

for src in df_train['src']:
    for char in src:
        if char not in char_to_id:
            char_to_id[char] = len(char_to_id)
            id_to_char[len(id_to_char)] = char

vocab_size = len(char_to_id)
print('Vocab size {}'.format(vocab_size))

Vocab size 18


# Data Preprocessing
 - The data is processed into the format required for the model's input and output.
 - Example: 1+2-3=0
     - Model input: 1 + 2 - 3 = 0
     - Model output: / / / / / 0 &lt;eos&gt;  (the '/' can be replaced with &lt;pad&gt;)
     - The key for the model's output is that the model does not need to predict the next character of the previous part. What matters is that once the model sees '=', it should start generating the answer, which is '0'. After generating the answer, it should also generate&lt;eos&gt;


In [6]:
def preprocess(df):
    char_id_lists = []
    label_id_lists = []

    for src in df['src']:
        char_id_list = []
        label_id_list = []
        
        flag = False
        for char in src:
            char_id_list.append(char_to_id[char])
            if char == '=':
                flag = True
            if flag and char != '=':
                label_id_list.append(char_to_id[char])
            elif char != '=':
                label_id_list.append(char_to_id['<pad>'])

        label_id_list.append(char_to_id['<eos>'])

        char_id_lists.append(char_id_list)
        label_id_lists.append(label_id_list)
    
    df['char_id_list'] = char_id_lists
    df['label_id_list'] = label_id_lists

    return df

In [7]:
df_train = preprocess(df_train)
df_eval = preprocess(df_eval)

In [8]:
df_train.head()

,Unnamed: 0,src,tgt,len,char_id_list,label_id_list
0,2285313,14*(43+20)=882,882,14,"[3, 4, 5, 6, 4, 7, 8, 9, 10, 11, 2, 12, 12, 9]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 9, 1]"
1,317061,(6+1)*5=35,35,10,"[6, 13, 8, 3, 11, 5, 14, 2, 7, 14]","[0, 0, 0, 0, 0, 0, 0, 7, 14, 1]"
2,718770,13+32+29=74,74,11,"[3, 7, 8, 7, 9, 8, 9, 15, 2, 16, 4]","[0, 0, 0, 0, 0, 0, 0, 0, 16, 4, 1]"
3,170195,31*(3-11)=-248,-248,14,"[7, 3, 5, 6, 7, 17, 3, 3, 11, 2, 17, 9, 4, 12]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 9, 4, 12, 1]"
4,2581417,24*49+1=1177,1177,12,"[9, 4, 5, 4, 15, 8, 3, 2, 3, 3, 16, 16]","[0, 0, 0, 0, 0, 0, 0, 3, 3, 16, 16, 1]"


In [9]:
df_eval.head()

,Unnamed: 0,src,tgt,len,char_id_list,label_id_list
0,2573208,48+43+34=125,125,12,"[4, 12, 8, 4, 7, 8, 7, 4, 2, 3, 9, 14]","[0, 0, 0, 0, 0, 0, 0, 0, 3, 9, 14, 1]"
1,1630340,30-(48+13)=-31,-31,14,"[7, 10, 17, 6, 4, 12, 8, 3, 7, 11, 2, 17, 7, 3]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 7, 3, 1]"
2,549277,(21*31)+10=661,661,14,"[6, 9, 3, 5, 7, 3, 11, 8, 3, 10, 2, 13, 13, 3]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 13, 13, 3, 1]"
3,133957,2-27-10=-35,-35,11,"[9, 17, 9, 16, 17, 3, 10, 2, 17, 7, 14]","[0, 0, 0, 0, 0, 0, 0, 17, 7, 14, 1]"
4,1279828,(15*20)+24=324,324,14,"[6, 3, 14, 5, 9, 10, 11, 8, 9, 4, 2, 7, 9, 4]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 9, 4, 1]"


# Hyper Parameters

|Hyperparameter|Meaning|Value|
|-|-|-|
|`batch_size`|Number of data samples in a single batch|64|
|`epochs`|Total number of epochs to train|10|
|`embed_dim`|Dimension of the word embeddings|256|
|`hidden_dim`|Dimension of the hidden state in each timestep of the LSTM|256|
|`lr`|Learning Rate|0.001|
|`grad_clip`|To prevent gradient explosion in RNNs, restrict the gradient range|1|

In [10]:
batch_size = 64
epochs = 2
embed_dim = 256
hidden_dim = 256
lr = 0.001
grad_clip = 1

# Data Batching
- Use `torch.utils.data.Dataset` to create a data generation tool called  `dataset`.
- The, use `torch.utils.data.DataLoader` to randomly sample from the `dataset` and group the samples into batches.

In [11]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    
    def __len__(self):
        # return the amount of data
        return len(self.sequences)
    
    def __getitem__(self, index):
        # Extract the input data x and the ground truth y from the data
        x = self.sequences.iloc[index]['char_id_list']
        y = self.sequences.iloc[index]['label_id_list']

        return x, y

# collate function, used to build dataloader
def collate_fn(batch):
    batch_x = [torch.tensor(data[0]) for data in batch]
    batch_y = [torch.tensor(data[1]) for data in batch]
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])
    
    # Pad the input sequence
    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens

In [12]:
ds_train = Dataset(df_train[['char_id_list', 'label_id_list']])
ds_eval = Dataset(df_eval[['char_id_list', 'label_id_list']])

In [13]:
# Build dataloader of train set and eval set, collate_fn is the collate function
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
dl_eval = torch.utils.data.DataLoader(ds_eval, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Model Design

## Execution Flow
1. Convert all characters in the sentence into embeddings.
2. Pass the embeddings through an LSTM sequentially.
3. The output of the LSTM is passed into another LSTM, and additional layers can be added.
4. The output from all time steps of the final LSTM is passed through a Fully Connected layer.
5. The character corresponding to the maximum value across all output dimensions is selected as the next character.

## Loss Function
Since this is a classification task, Cross Entropy is used as the loss function.

## Gradient Update
Adam algorithm is used for gradient updates.

In [14]:
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()
        
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])
        
        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.linear = torch.nn.Sequential(torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=hidden_dim),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=vocab_size))
        
    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)
    
    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        batch_x = self.embedding(batch_x)
        
        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)
        
        batch_x, _ = self.rnn_layer1(batch_x)
        batch_x, _ = self.rnn_layer2(batch_x)
        
        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x,
                                                            batch_first=True)
        
        batch_x = self.linear(batch_x)
        
        return batch_x
    
    def generator(self, start_char, max_len=200):
        
        char_list = [char_to_id[c] for c in start_char]
        
        next_char = None
        
        while len(char_list) < max_len: 
            # Write your code here 
            # Pack the char_list to tensor
            # Input the tensor to the embedding layer, LSTM layers, linear respectively

            input_tensor = torch.tensor(char_list).unsqueeze(0)
            input_length = torch.tensor([len(char_list)])

            y = self.forward(input_tensor, input_length)
            next_char = torch.argmax(y[0, -1, :]).item()
            
            if next_char == char_to_id['<eos>']:
                break
            
            char_list.append(next_char)
            
        return [id_to_char[ch_id] for ch_id in char_list]

In [15]:
torch.manual_seed(2)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

model = CharRNN(vocab_size,
                embed_dim,
                hidden_dim)

Using cuda device


In [16]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>'])
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

# Training
1. The outer `for` loop controls the `epoch`
    1. The inner `for` loop uses `data_loader` to retrieve batches.
        1. Pass the batch to the `model` for training.
        2. Compare the predicted results `batch_pred_y` with the true labels `batch_y` using Cross Entropy to calculate the loss `loss`
        3. Use `loss.backward` to automatically compute the gradients.
        4. Use `torch.nn.utils.clip_grad_value_` to limit the gradient values between `-grad_clip` &lt; and &lt; `grad_clip`.
        5. Use `optimizer.step()` to update the model (backpropagation).
2.  After every `1000` batches, output the current loss to monitor whether it is converging.

In [17]:
from tqdm import tqdm

In [18]:
model = model.to(device)

In [19]:
i = 0
for epoch in range(1, epochs+1):
    # The process bar
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    model.train()
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        # Write your code here
        # Clear the gradient

        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        batch_pred_y = model(batch_x, batch_x_lens)

        # Write your code here
        # Input the prediction and ground truths to loss function
        # Back propagation

        loss = criterion(batch_pred_y.view(-1, vocab_size), batch_y.view(-1))
        optimizer.zero_grad()
        loss.backward()

        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip) # gradient clipping

        # Write your code here
        # Optimize parameters in the model

        optimizer.step()

        i += 1
        if i % 50 == 0:
            bar.set_postfix(loss = loss.item())
    
    # Evaluate your model
    bar = tqdm(dl_eval, desc=f"Validation epoch {epoch}")
    matched = 0
    total = 0
    model.eval()

    cnt = 0
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        with torch.no_grad():

            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            batch_pred_y = model(batch_x, batch_x_lens)

            predictions  = torch.argmax(batch_pred_y, dim=-1)

            for pred, label, length in zip(predictions, batch_y, batch_y_lens):                
                pred = pred[:length]
                label = label[:length]

                if cnt < 3:
                    print(f"pred: {pred}")
                    print(f"label: {label}")

                if all(p == l for p, l in zip(pred, label) if l != char_to_id['<pad>']):
                    matched += 1

                total += 1
                cnt += 1

    print(f"matched: {matched}, total: {total}, accuracy: {matched/total}")

Validation epoch 1:   0%|          | 2/4114 [00:00<04:39, 14.70it/s]

pred: tensor([ 1,  1, 17,  4, 12, 15, 15,  3,  3,  9, 14,  1], device='cuda:0')
label: tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  3,  9, 14,  1], device='cuda:0')
pred: tensor([ 1,  1,  7,  7, 10,  1,  1,  1,  1,  1, 17,  7,  3,  1],
       device='cuda:0')
label: tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 17,  7,  3,  1],
       device='cuda:0')
pred: tensor([10,  1,  1, 17, 16, 17, 13, 13, 13, 13, 13, 16, 16,  1],
       device='cuda:0')
label: tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 13, 13,  3,  1],
       device='cuda:0')


Validation epoch 1: 100%|██████████| 4114/4114 [04:56<00:00, 13.89it/s]


matched: 153948, total: 263250, accuracy: 0.5847977207977207


Validation epoch 2:   0%|          | 2/4114 [00:00<03:58, 17.26it/s]

pred: tensor([ 1,  1, 17,  4, 12, 15,  3,  3,  3,  9, 14,  1], device='cuda:0')
label: tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  3,  9, 14,  1], device='cuda:0')
pred: tensor([ 1,  1,  7,  7, 10,  1,  1,  1,  1,  1, 17,  7,  3,  1],
       device='cuda:0')
label: tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 17,  7,  3,  1],
       device='cuda:0')
pred: tensor([10,  1,  1, 17, 13, 17, 17, 17, 13, 13, 13, 14,  7,  1],
       device='cuda:0')
label: tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 13, 13,  3,  1],
       device='cuda:0')


Validation epoch 2: 100%|██████████| 4114/4114 [04:31<00:00, 15.13it/s]

matched: 177859, total: 263250, accuracy: 0.6756277302943969


# Generation
Use `model.generator` and provide an initial character to automatically generate a sequence.

In [20]:
model = model.to("cpu")
print("".join(model.generator('1+1=')))

1+1=2
